# Data Structures and Algorithms in Python - Graph Algorithms (Part 1)
### AJ Zerouali, 2023/11/08

## 0) Introduction

**References:**

- Chapter 14 of "Data structures and algorithms in Python", by Goodrich, Tamassia and Goldwasser (abbreviated [GTG13]). More specifically sections 14.1-14.3.
- Section 18 of Portilla's "Python for Data Structures, Algorithms, and Interviews".
- Here we focus on the implementation of graphs as adjacecy lists and two search algos.

**Comments:**
- This is only a first sweep of this topic. Karimov's course goes in more depth than Portilla's.
- Portilla goes over the contents of sections 14.1-14.3 of [GTG13]. Karimov however also covers the algorithms discussed in Sections 14.5-14.7.
- In the first sweep, I will concentrate on sections 14.1-14.3 of [GTG13].
- I will skip Portilla's interview questions for section 18. There are infact only 3 questions:
    * Implement a graph.
    * Implement depth first search.
    * Implement breadth first search.
    Acoording to Portilla, these are the 3 basic interview questions for the material of section 18.
- 23/11/08: I will spend more time on this. There will be several parts of this notebook to correct/modify later.

## 1) Graphs as a mathematical structure

This part is based on [GTG13, Sec.14.1]. We start with the vocabulary of graphs.

### 1.a - Vocabulary

#### Definitions:

- A **graph** is a couple of sets $G=(V,E)$, where $V$ is the set of vertices, and where $E$ is the collection of edges $(u,v)$ connecting two vertices $u,v\in V$.
- If all edges in a graph are directed according to a (partial) order relation on $V$, we say that $G$ is a **directed graph** or a **digraph**. When all edges are undirected, $G$ is called an **undirected**, and if some edges are directed, we say that it's a **mixed graph**.
- If there is a weight attached to each edge in $E$, then $G$ is called a **weighted graph**.
- If $(u,v)\in E$ is an edge, $u$ is called the **origin** and $v$ is called the **destination** of the edge. When $u,v\in V$ are connected by an edge $(u,v)\in E$, we say that these vertices are **adjacent**.
- The **outgoing** edges of a vertex $u\in V$ are the edges $\alpha \in E$ whose origin is $u$. Similarly, the **incoming** edges of $u\in V$ are the $\beta \in E$ having $u$ as a destination. An edge $e\in E$ is said to be **incident** to a vertex $u\in V$ if $v$ is one of the endpoints of $e$.
- The **degree** $deg(u)$ of a vertex $u\in V$ is the number of incident edges to $u$. Respectively, the number of incoming and outgoing edges to $u\in V$ are the **in-degree** and **out-degree**, denoted by $indeg(u)$ and $outdeg(u)$.
- A graph $G=(V,E)$ is said to be **simple** if there are no parallel edges between the same vertices, and if there are no self-loops (i.e. edges connecting a vertex to itself).
- A **path** is a sequence of alternating vertices and edges $(v_1,e_1,v_2,e_2,\cdots,v_n)$ such that each $e_i\in E$ is incident to both $v_i,v_{i+1}\in V$. When all edges in a path are directed, we say that the path is directed.
- A path that starts and end at the same vertex $v_1=v_n$ is called a **cycle**. When all vertices of a cycle other than the origin are distinct, we say that the cycle is **simple**. A graph that contains no cycles is said to be **acyclic**.


#### Comments:
1) Here, it's important to underline that we described the edges $E$ of a graph $G=(V,E)$ as a collection and not as a set. This is to allow several edges connecting two vertices. In the case where $G$ is simple, $E$ is indeed a well-defined set.
2) Unless otherwise stated, [GTG13] restrict to simple graphs in chapter 14.

The next terms are discussed after Example 14.6 of [GTG13]. Suppose $G = (V,E)$ is a graph.
* Given two vertices $u,v \in V$, we say that $v$ is **reachable** from $u$ if there exists a path in $G$ that connects these two vertices. In the case where $G$ is directed, reachability requires the existence of a directed path, meaning that this is not a symmetric relation in general. For undirected graphs however, reachability is symmetric.
* The graph $G$ is said to be **connected** if for any vertices $u,v\in V$ there exists a path from $u$ to $v$.
* A directed graph is **strongly connected** if for any vertices $u,v\in V$, $v$ is reachable from $u$ and $u$ is reachable from $v$.
* A **subgraph** $H=(V_H, E_H)$ of $G$ is a graph such that $V_H\subseteq V$ and $E_H\subseteq E$. A **spanning subgraph** $H$ of $G$ is a subgraph that contains all the vertices of $G$.
* If a graph is not connected, its maximal connected subgraphs are called **connected components**.
* A **forest** is a graph without cycles. A connected forest is called a **tree**.
* A **spanning tree** $T $of a graph $G$ is a connected, spanning, acyclic subgraph.

Note that in these last definitions, the trees are slightly different from the ones previously seen, as there are no designated root.



### 1.b - Some graph properties

In the sequel, we assume that $G$ is a simple graph.

#### Proposition 14.8
If $G = (V,E)$ has $m$ edges, then: 
$$\sum_{v\in V}deg(v) = 2m.$$

#### Proof:
For every $v\in V$, write:
$$O_v = \{e\in E \ |\ e = (v,w), w\in V\},$$
$$D_v = \{e\in E \ |\ e = (u,v), u\in V\},$$
and allowing these subsets to be empty. Notice then that $E = \sqcup_{v\in V}O_v = \sqcup_{v\in V}D_v$, so that:
$$m = \mathrm{card}(E) = \sum_{v\in V}\mathrm{card}(O_v)= \sum_{v\in V}\mathrm{card}(D_v).$$
Since by definition $deg(v)=\mathrm{card}(O_v)+\mathrm{card}(D_v)$, summing the latter yields:
$$\sum_{v\in V}deg(v) = 2\mathrm{card}(E)=2m.$$

#### Proposition 14.9
If $G = (V,E)$ is directed with $m$ edges, then: 
$$ \sum_{v\in V}indeg(v) = \sum_{v\in V}outdeg(v) = m.$$

#### Proof:
Using the notation of the previous proof, we have by definition that $indeg(v)=\mathrm{card}(O_v)$ and $indeg(v)=\mathrm{card}(D_v)$. The result follows by summing over the vertices.

#### Proposition 14.10
If $G = (V,E)$ is simple and has $m$ edges and $n$ vertices, then:
- If $G$ is directed then $m\le n(n-1)$.
- If $G$ is undirected then $m\le n(n-1)/2$.

#### Proof:
Given $G=(V,E)$, we can complete it to a connected graph $\tilde{G}=(V,\tilde{E})$ by adding edges connecting each $v\in V$ to all other $w\in V\smallsetminus\{v\}$. In this case, we'll obviously have $m\le \mathrm{card}(\tilde{E})$, with $\mathrm{card}(\tilde{E})$ being the **highest upper-bound possible** for $\mathrm{card}(E)$. Now, there are two cases:
- If $G$ is simple and undirected, then for all $v\in V$ and $w\in V\smallsetminus\{v\}$, we have $(v,w)=(w,v)\in \tilde{E}$. Next, summing the number of edges **not counted yet** for each vertex in $V$, we have:
$$\mathrm{card}(\tilde{E})=\sum_{i=1}^n(n-i)=\sum_{j=0}^{n-1} j = n(n-1)/2.$$
- If $G$ is simple and directed, then for all $v\in V$ and $w\in V\smallsetminus\{v\}$, we have $(v,w)\ne (w,v)\in \tilde{E}$. As such, the cardinality of $\tilde{E}$ for a directed $G$ is twice that of the undirected case.
 


## 2) Implementations of graphs

### 2.a - Implementation as an adjacency list

This part follows lecture 155 of Portilla's course. Link to notebook:

https://github.com/jmportilla/Python-for-Algorithms--Data-Structures--and-Interviews/tree/master/08-Graphs

The starting point of this section is the adjacency matrix associated to a (weighted) graph. This notion is dicussed in Lecture 154. For graphs with small numbers of vertices and/or edges, the adjacency matrix is rather sparse, meaning that it is not a convenient structure for the implementation of small graphs. This issue can be resolved by resorting to **adjacency lists**, which store each vertex of the graph, and assign a dictionary to each vertex to store the connections with other vertices.

To represent a graph then, we will create two classes:
1) The *Vertex* class, which will represent the individual vertices of the graph, and store a dictionary whose keys are the other vertices to which it is connected, and whose values are the weights of the resulting edges. So if *Vertex* represents some $v\in V$ for a given $G=(V,E)$, then the object representing $v$ stores the subsets $O_v$ and $D_v$ of the previous section.
2) The *Graph* class, which represents the graph $G$. Since $E$ is essentially represented by the vertices, the *Graph* object will just keep a master list of vertices (the adjacency list) representing $V$.

For the interface of the *Vertex* class, we want the following attributes:
* **id**: The string storing the name of the vertex.
* **connectedTo:** The dictionary of weighted edges.

For the methods of *Vertex*, we'll implement:
* **getId()**: Returns ID of current vertex.
* **__ str __()**: Overloading of the *str* function to display the *connectedTo* contents.
* **addNeighbor(vertex)**: To add a connection from current vertex to another. 
* **getConnections()**: Which returns all of the vertices in the adjacency list, as represented by the connectedTo instance variable. 
* **getWeight(vertex)**: Which returns the weight of the edge from this vertex to the vertex passed as a parameter.




In [13]:
class Vertex:
    def __init__(self, vtx_id):
        self.id = vtx_id
        self.connectedTo = {}
    
    def getId(self):
        return self.id
    
    def addNeighbor(self, vtx, weight = 0):
        self.connectedTo[vtx] = weight
    
    def __str__(self):
        return "Vertex "+str(self.id)+ " is connected to: "+str([x.id for x in self.connectedTo])
    
    def getConnections(self):
        return self.connectedTo.keys()
    
    def getWeight(self, vtx):
        return self.connectedTo[vtx]

Next, the interface of the *Graph* class will consist of the master list and number of vertices as attributes, along with the following methods:
* **addVertex(vert)**: Which adds an instance of Vertex to the graph.
* **addEdge(fromVert, toVert, weight)**: Which adds a new, weighted, directed edge to the graph that connects two vertices.
* **getVertex(vertKey)**: To find the vertex in the graph named vertKey.
* **getVertices()**: Which returns the list of all vertices in the graph.
* **__ contains __**: To return True for a statement of the form "vertex in graph", if the given vertex is in the graph, False otherwise.

In [26]:
class Graph:
    def __init__(self):
        self.vertices = {}
        self.N_vertices = 0
    
    def addVertex(self, vtx_id):
        self.vertices[vtx_id] = Vertex(vtx_id)
        self.N_vertices += 1
        return self.vertices[vtx_id]
    
    def addEdge(self, orgn_vtx, dest_vtx, weight=0):
        '''
            This implementation only adds outgoing connections
        '''
        if orgn_vtx not in self:
            nv = self.addVertex(orgn_vtx)
        if dest_vtx not in self:
            nv = self.addVertex(dest_vtx)
        self.vertices[orgn_vtx].addNeighbor(self.vertices[dest_vtx], weight)
        # If we want to also consider incoming vertices
        #self.vertices[dest_vtx].addNeighbor(self.vertices[orgn_vtx], -weight)
        
    
    def getVertex(self, vtx_id):
        if vtx_id in self.vertices:
            return self.vertices[vtx_id]
        else:
            return None
    
    def getVertices(self):
        return list(self.vertices.keys())
    
    def __contains__(self, vtx):
        return vtx in self.vertices
    
    def __iter__(self):
        return iter(self.vertices.values())
    

Let's test these classes.

In [15]:
g = Graph()

In [16]:
vert_dict = {"A":6, "B": 3, "C":8 , "D": 7, "E":9, "F":1}

In [17]:
for k in vert_dict:
    g.addVertex(k)

In [18]:
g.addEdge("A", "B", 0.75)
g.addEdge("A", "C", 0.25)
g.addEdge("C", "D", 0.35)
g.addEdge("C", "E", 0.65)
g.addEdge("B", "F", 1.0)


In [19]:
v = g.getVertex("B")
v.getConnections()

dict_keys([<__main__.Vertex object at 0x7f55d5535040>, <__main__.Vertex object at 0x7f55d5149c70>])

In [21]:
print(v)

Vertex B is connected to: ['A', 'F']


In [22]:
v.connectedTo

{<__main__.Vertex at 0x7f55d5535040>: -0.75,
 <__main__.Vertex at 0x7f55d5149c70>: 1.0}

### 2.b - A more sophisticated implementation

This part follows lecture 160 of Portilla's course. The notebook can be found here:

https://github.com/jmportilla/Python-for-Algorithms--Data-Structures--and-Interviews/blob/master/08-Graphs/01-Implementation%20of%20Graph%20Overview.ipynb

For the implementation of the traversal algorithms, it will be useful to color the vertices and avoid revisiting the same vertex, and/or add a verification list for visits.

We will also use the *OrderedDict* and *Enum* tools of Python. In brief:
- *OrderedDict* assigns an insertion order for the keys, and otherwise is a dictionary.
- *Enum* is used as an abstract class, in which we declare names and values assigned to each state.


In [1]:
from collections import OrderedDict
from enum import Enum

In [6]:
class State(Enum):
    unvisited = 0
    visiting = 1
    visited = 2

class Node:
    
    def __init__(self, num):
        self.num = num
        self.visit_state = State(0)
        self.adjacent = OrderedDict() # Key = None and value will be the weight
        
    def __str__(self):
        return str(self.num)

class Graph:
    
    def __init__(self):
        self.nodes = OrderedDict()
    
    def add_node(self, num):
        self.nodes[num] = Node(num)
        return self.nodes[num]
    
    def reset_visit_states(self):
        for node in self.nodes:
            node.visit_state = State(0)
    
    def add_edge(self, source, dest, weight = 0):
        
        if source not in self.nodes:
            self.add_node(source)
            
        if dest not in self.nodes:
            self.add_node(dest)
            
        self.nodes[source].adjacent[self.nodes[dest]] = weight
    

Now we test this implementation:

In [7]:
g = Graph()

In [11]:
g.add_edge(0,1,5)
g.add_edge(1,3,10)

In [12]:
g.nodes

OrderedDict([(0, <__main__.Node at 0x7f55161bde20>),
             (1, <__main__.Node at 0x7f55161bdc40>),
             (3, <__main__.Node at 0x7f55160b8a60>)])

In [10]:
g.nodes[0].adjacent

OrderedDict([(<__main__.Node at 0x7f55161bdc40>, 5)])

We will use this implementation of a graph in the next sections.

## 3) Breadth First Seach

This is the first graph traversal algorithm that we'll discuss. This section is based on Lectures 156, 157, and 162 of Portilla's course, as well as [GTG13, Sec.14.3.3].

### 3.a - Motivation: Word ladder problem

The word ladder problem consists of transforming one word into another one with the same number of letters, such that
- At each step, only one letter is modified.
- Each time a letter is modified, the resulting string is still a word.

As an example, if we consider a collection of 4-letter words, and say we'd like to transform "FOOL" into "SAGE". An example sequence in this case is: FOOL->POOL->POLL->POLE->SOLE->SALE->SAGE.

The word ladder problem can be solved with a graph traversal algorithm. In this setting, we consider the vertices of our graph as 4-letter words, with two words being related by an edge if and only if they differ by precisely one letter.

To illustrate our present example, it will be useful to color our vertices as black, white, or gray, which will allow us to write a graph traversal algorithm

In [27]:
class Vertex:
    def __init__(self, vtx_id):
        self.id = vtx_id
        self.connectedTo = {}
    
    def getId(self):
        return self.id
    
    def addNeighbor(self, vtx, weight = 0):
        self.connectedTo[vtx] = weight
    
    def __str__(self):
        return "Vertex "+str(self.id)+ " is connected to: "+str([x.id for x in self.connectedTo])
    
    def getConnections(self):
        return self.connectedTo.keys()
    
    def getWeight(self, vtx):
        return self.connectedTo[vtx]
    
class Graph:
    def __init__(self):
        self.vertices = {}
        self.N_vertices = 0
    
    def addVertex(self, vtx_id):
        self.vertices[vtx_id] = Vertex(vtx_id)
        self.N_vertices += 1
        return self.vertices[vtx_id]
    
    def addEdge(self, orgn_vtx, dest_vtx, weight=0):
        '''
            This implementation only adds outgoing connections
        '''
        if orgn_vtx not in self:
            nv = self.addVertex(orgn_vtx)
        if dest_vtx not in self:
            nv = self.addVertex(dest_vtx)
        self.vertices[orgn_vtx].addNeighbor(self.vertices[dest_vtx], weight)
        # If we want to also consider incoming vertices
        #self.vertices[dest_vtx].addNeighbor(self.vertices[orgn_vtx], -weight)
        
    
    def getVertex(self, vtx_id):
        if vtx_id in self.vertices:
            return self.vertices[vtx_id]
        else:
            return None
    
    def getVertices(self):
        return list(self.vertices.keys())
    
    def __contains__(self, vtx):
        return vtx in self.vertices
    
    def __iter__(self):
        return iter(self.vertices.values())

In [1]:
from collections import OrderedDict
from enum import Enum

class State(Enum):
    unvisited = 0
    visiting = 1
    visited = 2

class Node:
    
    def __init__(self, num):
        self.num = num
        self.visit_state = State(0)
        self.adjacent = OrderedDict() # Key = None and value will be the weight
        
    def __str__(self):
        return str(self.num)

class Graph:
    
    def __init__(self):
        self.nodes = OrderedDict()
    
    def add_node(self, num):
        self.nodes[num] = Node(num)
        return self.nodes[num]
    
    def reset_visit_states(self):
        for node in self.nodes:
            node.visit_state = State(0)
    
    def add_edge(self, source, dest, weight = 0):
        
        if source not in self.nodes:
            self.add_node(source)
            
        if dest not in self.nodes:
            self.add_node(dest)
            
        self.nodes[source].adjacent[self.nodes[dest]] = weight
    

Next, we build a graph from a collection of words. We do this using a function that will put words into buckets, create vertices, and then add edges depending on whetehr or not the vertices are related.

In [2]:
# Word list (graph vertices)
words = ["fool", "foul", "foil", "fail", "fall",
         "pall", "poll", "pole", "cool", "pope",
         "pale", "sale", "page", "sage", "paul",
         "pool"
        ]

# Graph builder function
def buildGraph(word_list):
    d = {}
    g = Graph()
    
    # create buckets of words that differ by one letter
    for word in word_list:
        print(word)
        for i in range(len(word)):
            bucket = word[:i] + '_' + word[i+1:]
            if bucket in d:
                d[bucket].append(word)
            else:
                d[bucket] = [word]
    # add vertices and edges for words in the same bucket
    for bucket in d.keys():
        for word1 in d[bucket]:
            for word2 in d[bucket]:
                if word1 != word2:
                    #g.addEdge(word1,word2)
                    g.add_edge(word1,word2)
    return g

word_graph = buildGraph(words)

fool
foul
foil
fail
fall
pall
poll
pole
cool
pope
pale
sale
page
sage
paul
pool


In [38]:
for w in word_graph.vertices.keys():
    print(word_graph.vertices[w])

Vertex fool is connected to: ['cool', 'pool', 'foul', 'foil']
Vertex cool is connected to: ['fool', 'pool']
Vertex pool is connected to: ['fool', 'cool', 'poll']
Vertex foul is connected to: ['fool', 'foil']
Vertex foil is connected to: ['fool', 'foul', 'fail']
Vertex fail is connected to: ['foil', 'fall']
Vertex fall is connected to: ['fail', 'pall']
Vertex pall is connected to: ['fall', 'poll', 'paul', 'pale']
Vertex poll is connected to: ['pall', 'pool', 'pole']
Vertex paul is connected to: ['pall']
Vertex pale is connected to: ['pall', 'pole', 'sale', 'page']
Vertex pole is connected to: ['poll', 'pale', 'pope']
Vertex pope is connected to: ['pole']
Vertex sale is connected to: ['pale', 'sage']
Vertex page is connected to: ['pale', 'sage']
Vertex sage is connected to: ['sale', 'page']


In [5]:
word_graph.nodes

OrderedDict([('fool', <__main__.Node at 0x7f0fac6edc70>),
             ('cool', <__main__.Node at 0x7f0fac6edcd0>),
             ('pool', <__main__.Node at 0x7f0fac6edf70>),
             ('foul', <__main__.Node at 0x7f0fac6eddc0>),
             ('foil', <__main__.Node at 0x7f0fac6edb20>),
             ('fail', <__main__.Node at 0x7f0fac699730>),
             ('fall', <__main__.Node at 0x7f0fac699280>),
             ('pall', <__main__.Node at 0x7f0fac699790>),
             ('poll', <__main__.Node at 0x7f0fac699c10>),
             ('paul', <__main__.Node at 0x7f0fac699c40>),
             ('pale', <__main__.Node at 0x7f0fac6993d0>),
             ('pole', <__main__.Node at 0x7f0fac699370>),
             ('pope', <__main__.Node at 0x7f0fac6a4820>),
             ('sale', <__main__.Node at 0x7f0fac6a4370>),
             ('page', <__main__.Node at 0x7f0fac6a4d60>),
             ('sage', <__main__.Node at 0x7f0fac6a4d00>)])

In [6]:
for x in word_graph.nodes["pool"].adjacent.keys():
    print(x)

fool
cool
poll


In [9]:
for w in word_graph.nodes.keys():
    print("\'"+str(word_graph.nodes[w])+"\'"+" is connected to: "+ str([x.num for x in word_graph.nodes[w].adjacent.keys()]))

'fool' is connected to: ['cool', 'pool', 'foul', 'foil']
'cool' is connected to: ['fool', 'pool']
'pool' is connected to: ['fool', 'cool', 'poll']
'foul' is connected to: ['fool', 'foil']
'foil' is connected to: ['fool', 'foul', 'fail']
'fail' is connected to: ['foil', 'fall']
'fall' is connected to: ['fail', 'pall']
'pall' is connected to: ['fall', 'poll', 'paul', 'pale']
'poll' is connected to: ['pall', 'pool', 'pole']
'paul' is connected to: ['pall']
'pale' is connected to: ['pall', 'pole', 'sale', 'page']
'pole' is connected to: ['poll', 'pale', 'pope']
'pope' is connected to: ['pole']
'sale' is connected to: ['pale', 'sage']
'page' is connected to: ['pale', 'sage']
'sage' is connected to: ['sale', 'page']


**Comment:** I got this code from https://github.com/jmportilla/Python-for-Algorithms--Data-Structures--and-Interviews/blob/master/08-Graphs/03-Word%20Ladder%20Example%20Problem.ipynb. 

### 3.b - The Breadth First Search (BFS) algorithm

This part is based on [GTG13, Sec.14.3.3] and on Lecture 162 of Portilla's course.

The most important property of BFS is that it returns the shortest path between two vertices if there exists one.


In [30]:
from collections import deque

In [33]:
def BFS(G, start):
    visited = set()
    queue = deque([start])
    
    while len(queue)>0:
        node = queue.popleft()
        if node not in visited:
            visited.add(node)
            queue.extend(list(G.nodes[node].adjacent.keys())-visited)
    
    return visited

In [34]:
BFS(word_graph, word_graph.nodes["fool"])

KeyError: <__main__.Node object at 0x7f478a944f40>

**Comment (23/11/08):**

I'm moving on. Portilla jumped from his implementation of a graph to an implementation of BFS/DFS that do not use that class. It's annoying, and I will fix these errors another time (there's still a lot to learn with graphs).

### 3.c - Some properties of BFS

Here, we suppose for simplicity that $G$ is a connected graph in the topological sense.

#### Proposition 14.16:
Let $G=(V,E)$ be a directed or undirected graph, on which a BFS traversal starting at $v_0\in V$ has been performed. Then:
1) BFS visits all vertices of $G$ that are reachable from $v_0$.s
2) For each vertex $v\in V$ at level $i$, the path of the BFS tree $T$ between $v_0$ and $v$ has $i$ edges.
3) If $(u,v)$ is an edge that is not in the BFS tree, then the level number of $v$ is at most $1$ plus the level number of $v$.

#### Proposition 14.17:
Suppose $G=(V,E)$ is represented with an adjacency list structure. The time complexity of the BFS algorithm is $O\left(\mathrm{card}(V)+ \mathrm{card}(E) \right)$.

## 4) Depth First Seach

This is the second graph traversal algorithm of this notebook. This section is based on Lectures 158, 159, 161 of Portilla's course, as well as [GTG13, Sec.14.3.1-14.3.2].

### 4.a - Motivation: Knight's tour problem

The notebook for this exercise is here: https://github.com/jmportilla/Python-for-Algorithms--Data-Structures--and-Interviews/blob/master/08-Graphs/04-Knight's%20Tour%20Example%20Problem.ipynb.

The knight tour problem is described as follows:
- We have a chess board (8x8) on which there is only the knight at one of the corners.
- We want to visit each square of the board exactly once. Here, it's important to note that the knight can only move either by 2 squares vertically then 1 horizontally, or two squares vertically and then 1 horizontally. By "visiting" a square, we mean the square reached by at the end of the move, and we don't count the squares *during* the move.

This problem can be solved by a **depth first** graph traversal. The idea is to model all the squares of the graph as vertices, with edges corresponding to admissible knight moves.

In rough terms, starting from a given square on the board, depth first traversal builds an exploration tree whose root is the initial square, and the children of the root are the squares that can be reached in one move. Continuing this process recursively, depth first search first explores how deep the path in the tree could be (or how long it could be in the graph), and backtracks if the last vertex reached is a dead end. Unlike breadth first search, it is a recursive algorithm, and generalizes the inorder, preorder and postorder traversals studied for binary trees.

**Comment:** 

Instead of redoing Portilla's implementation from scratch for the knight's tour problem, I will focus on the general implementation of the DFS algorithm.


### 4.b - Depth First Search (DFS)

This subsection is based on [GTG13, Sec.14.3.1] and Lecture 161 of Portilla's DSA course.

We start with an analogy. Depth first search is similar to exploring a labyrinth with a string and a can of paint. The idea is to proceed recursively, starting from an input node $s$. At the current vertex $u$:
- Consider an arbitrary edge $(u,v)$. 
- If the vertex $v$ is already painted (i.e. has already been visited), backtrack to $u$.
- If the vertex $v$ has not been visited yet, unenroll the string, visit $v$, and paint it to designate it as visited. 
- Repeat the above with the next edge $(u',v')$, until all the incident edges to the current vertex lead to vertices that have all been visited.

DFS is typically implemented as a recursion, as the following pseudo-code shows:

        Algorithm DFS(G,u):
            Input: A graph G and a vertex u therein.
            Output: Collection of vertices reachable from u, along with discovery edges.
            
            for each outgoing edge e = (u,v) of u:
                if v has not been visited:
                    Mark v as visited
                    call DFS(G,v)

Some observations are relevant at this point:
- The DFS algorithm builds a depth first search tree rooted at the starting vertex $s$.
- A **discovery (or tree) edge** $e=(u,v)$ is an edge leading to a non-visited vertex $v$ in the execution of DFS. All other edges considered, which lead to previously visited vertices, are called **non-tree edges**.

Based on the graph implementation of section 2.b above, we can implement DFS as follows:

In [27]:
def DFS(G, u, dfs_tree = None):
    
    if dfs_tree is None:
        dfs_tree = OrderedDict() 
        dfs_tree[u.num]= []
    
    for v in u.adjacent.keys():
        if v.visit_state.value == 0:
            v.visit_state = State(2)
            dfs_tree[u.num].append(v.num)
            dfs_tree[v.num] = []
            DFS(G,v, dfs_tree)
    
    return dfs_tree

#### Test:

In [14]:
from collections import OrderedDict
from enum import Enum

class State(Enum):
    unvisited = 0
    visiting = 1
    visited = 2

class Node:
    
    def __init__(self, num):
        self.num = num
        self.visit_state = State(0)
        self.adjacent = OrderedDict() # Key = None and value will be the weight
        
    def __str__(self):
        return str(self.num)

class Graph:
    
    def __init__(self):
        self.nodes = OrderedDict()
    
    def add_node(self, num):
        self.nodes[num] = Node(num)
        return self.nodes[num]
    
    def reset_visit_states(self):
        for node in self.nodes:
            self.nodes[node].visit_state = State(0)
    
    def add_edge(self, source, dest, weight = 0):
        
        if source not in self.nodes:
            self.add_node(source)
            
        if dest not in self.nodes:
            self.add_node(dest)
            
        self.nodes[source].adjacent[self.nodes[dest]] = weight
    

In [24]:
# Word list (graph vertices)
words = ["fool", "foul", "foil", "fail", "fall",
         "pall", "poll", "pole", "cool", "pope",
         "pale", "sale", "page", "sage", "paul",
         "pool"
        ]

# Graph builder function
def buildGraph(word_list):
    d = {}
    g = Graph()
    
    # create buckets of words that differ by one letter
    for word in word_list:
        print(word)
        for i in range(len(word)):
            bucket = word[:i] + '_' + word[i+1:]
            if bucket in d:
                d[bucket].append(word)
            else:
                d[bucket] = [word]
    # add vertices and edges for words in the same bucket
    for bucket in d.keys():
        for word1 in d[bucket]:
            for word2 in d[bucket]:
                if word1 != word2:
                    #g.addEdge(word1,word2)
                    g.add_edge(word1,word2)
    return g

word_graph = buildGraph(words)

fool
foul
foil
fail
fall
pall
poll
pole
cool
pope
pale
sale
page
sage
paul
pool


In [28]:
word_graph.reset_visit_states()
dfs_tree = DFS(word_graph, word_graph.nodes["fool"])

In [29]:
dfs_tree

OrderedDict([('fool', ['pool']),
             ('cool', ['fool']),
             ('pool', ['poll']),
             ('poll', ['pall']),
             ('pall', ['fall', 'paul', 'pale']),
             ('fall', ['fail']),
             ('fail', ['foil']),
             ('foil', ['foul']),
             ('foul', []),
             ('paul', []),
             ('pale', ['pole', 'sale']),
             ('pole', ['pope']),
             ('pope', []),
             ('sale', ['sage']),
             ('sage', ['page']),
             ('page', [])])

### 4.c - Properties of DFS

The results presented here are from [GTG13, Sec.14.3.1].

#### Proposition 14.12:
Let $G=(V,E)$ be an undirected graph on which a depth first traversal is performed, starting from $s\in V$. Then:
- DFS visits all the vertices in the connected component of $G$ containing $s$.
- The discovery edges obtained from DFS constitute a spanning tree of the connected component of $s$.

#### Proposition 14.13:
Let $G=(V,E)$ be a directed graph on which a depth first traversal is performed, starting from $s\in V$. Then:
- DFS visits all the vertices of $G$ reachable from $s$.
- The resulting DFS tree contains directed paths from $s$ to every vertex reachable from $s$.

#### Proposition 14.14-15:
Let $G=(V,E)$ be a directed or undirected graph. Then DFS traversal of $G$ can be performed in $O\left( \mathrm{card}(V)+\mathrm{card}(E)\right)$ time.

The justifications for this last part is discussed in the remainder of [GTG13, Ch.14].